## Machine Learning - CS60050
### Roll No: 19EC30055
### Name: Ujwal Nitin Nayak
### Assignment No: 2

### Introduction

- The K-Nearest Neighbours algorithm is a non-parametric supervised learning algorithm which can be used both as classifier and a regressor. This algorithm assumes that two instances are similar if they exist in close proximity. 
- Unlike a decision tree, it does not use the training data to build a model and there is no requirement to tune several parameters or make any assumptions. 
- For a classification problem, every time a test example is encountered, the proximity or distance of this example to the training set examples is calculated, the k closest neighbours are chosen and then weights are assigned to each training example. This weight is typically an inverse distance weighting. The weights corresponding to each class are then summed up and the class with the highest summed weight is used to label the test example.
- Proximity between a training example and a test example can be found using one of several distance metrics namely Minkowski, Manhattan, Euclidean, Chebyshev, Mahalanobis, etc. In this assignment, we will use the Euclidean distance.

#### Importing Required Libraries

In [ ]:
import numpy as np #For mathematical functions
import pandas as pd #For handling imported data

#### Importing Training Data and Storing Attribute Names

In [ ]:
df=pd.read_csv('project2.csv')
attributes=df.columns.values.tolist();
df.head(10)

#### Storing the Target Labels and Dropping the Target Column from the Training DataFrame

In [ ]:
labels=df.iloc[:,len(attributes)-1].values;
df=df.drop(columns=['target']);
df.head(10)

#### Importing Test Data

In [ ]:
df_test=pd.read_csv('project2_test.csv');
df_test.head(4)

#### Standardization of Data using Z-Scores

Since the KNN algorithm is a distance-based algorithm, it is very prone to errors due to different ranges of features. Feature scaling using standardization ensures that all the values are centred around the mean with unit standard deviation, that is, the mean of the attribute becomes zero and the resultant distribution has a unit standard deviation. 
Feature scaling is done using the functions standardize_train and standardize_test. 
The standardize_train function takes the training data as its argument and finds the mean and standard deviation of each column, subtracts this mean from each column entry and then divides the entries by the standard deviation. The standardize_test functions uses the mean and standard deviation obtained for each of the columns of the training set and uses these to standardize the test set examples.

In [ ]:
#Standardizing Training Data
def standardize_train(df):
    norm_vals=[];
    #Looping over columns
    for col in df.columns:
        mean=df[col].mean()           #Finding mean
        std=df[col].std()             #Finding standard deviation
        #Standardizing and storing mean and standard deviation values for test set standardization
        df[col]=(df[col]-mean)/std;   
        norm_vals.append(np.array([mean,std])); 
    return norm_vals;

In [ ]:
#Standardizing Test Data
def standardize_test(df_test,norm_vals):
    cnt=0;
    #Using mean and standard deviation of corresponding column of training set to standardize test set
    for col in df_test.columns:
        mean=norm_vals[cnt][0];
        std=norm_vals[cnt][1];
        df_test[col]=(df_test[col]-mean)/std;
        cnt+=1;

In [ ]:
#Calling standardize functions
norm_vals=standardize_train(df);
standardize_test(df_test,norm_vals);
df.head(10)

In [ ]:
df_test.head(4)

In [ ]:
#Storing the values of each row of the dataframes into numpy arrays
examples=df.values;
test_examples=df_test.values;

In [ ]:
print(examples.shape)
print(test_examples.shape)
print(labels.shape)

#### Euclidean Distance

The function euclidean is used to calculate the Euclidean distance between a row of the training and the test sets. 
The input arguments are row_test and row_train. First, the squared differences between corresponding attribute values of row_test and row_train are calculated and then summed over all attributes. The square root of this sum gives the distance between the examples. 

In [ ]:
def euclidean(row_test,row_train):
    distance=np.sum((row_test-row_train)**2);
    return np.sqrt(distance);

#### Finding the Nearest Neighbours

Nearest neighbours are found using the find_nearest_neighbours function. This function takes the training examples, one row of the test examples and the value of k as its arguments and performs the following:
- Calls the euclidean function to find the distance between every training example and the current test example.
- Stores the index of the training example and the distance in a list.
- Sorts the list in increasing order based on the distance value.
- Returns the first k elements of the sorted list of neighbours.

In [ ]:
def find_nearest_neighbours(examples,entry,k):
    neighbours=[];
    idx=0; #Variable to keep track of the index
    #Looping over all rows in the training example
    for row in examples:
        #Calling euclidean function to find distance 
        dist=euclidean(entry,row);
        #Appending index and distance to the list
        neighbours.append((idx,dist));
        #Incrementing the index
        idx+=1;
    #Sorting based on the second parameter (dist)
    neighbours.sort(key=lambda x:x[1]);
    #Slicing first k elements
    nearest_neighbours=neighbours[:k];
    return nearest_neighbours;

#### Assigning Weights

Weights are assigned to the k nearest neighbours using the get_weights function which takes these neighbours and the number of training examples as its arguments. It does the following:
- Initialises an empty list of weights with size equal to the number of training examples.
- Loops over all the nearest neighbours and assigns (1/distance)^2 weight to the training example having a given index.
- Returns the list of weights

In [ ]:
def get_weights(neighbours,sz):
    weights=[None]*sz;
    for idx,dist in neighbours:
        weights[idx]=(1/(dist**2));
    return weights;

#### KNN Driver Function

This function takes the training examples, corresponding labels and the test examples as its arguments and does the following:
- Sets the value of k (through user input alongwith a default value) for the find_nearest_neighbour function. While there is no optimal way to choose the value of k, a rule of thumb is to choose floor(sqrt(n)) where n is the the number of training examples. If this comes out to be even, it is usually incremented or decremented by 1. In this assignment, incrementation is chosen (but for the given training examples the if block is not used since floor(sqrt(299)) = 17 which is odd).
- Obtains the k closest neighbours using the find_nearest_neighbour function and uses the index parameter of the  index,distance pair to find the corresponding labels. 
- Obtains the weights corresponding to these indices using the get_weights function and then finds the weighted distance for each of the possible output values
- Sorts the list of weighted distances in decreasing order based on the sum_weights parameter and stores the label with highest weighted distance as the prediction for that particular row.
- Returns the list of predictions

In [ ]:
def knn(examples,labels,test_examples):
    #Taking k as user input
    k=input("Enter the value of k (1-" +str(len(examples))+") (enter 0 to use default value) ");
    k=int(np.floor(float(k)));
    #Setting default value of k as the floor(sqrt(no_of_training_examples))
    if (k<0 or k>len(examples)):
        print('Value outside range, taking default value');
        k=0;
    if(k==0):
        k=int(np.floor(np.sqrt(len(examples))));
        if(k%2==0):
            k=k+1;
    print('Value of k = ',end="");
    print(k);
    #Defining the list of predictions
    predictions=list();
    #Looping over each row of the test set
    for entry in test_examples:
        #Finding the k nearest neighbours
        neighbours=find_nearest_neighbours(examples,entry,k);
        #Storing labels corresponding to these neighbours in a list
        outputs=[labels[idx] for idx,dist in neighbours];
        #Finding the unique values and corresponding counts using the above list
        values,counts=np.unique(outputs,return_counts=True);
        #Obtaining the weights correspoding to the training examples
        weights=get_weights(neighbours,len(examples));
        #Defining the list of weighted distances for each class label
        weighted_distances=list();
        #Looping over all unique label values
        for ind in values:
            #Looping over neighbours and summing weights for training examples 
            #having the class label of interest
            sum_weights=0.0;
            for idx,dist in neighbours:
                if labels[idx]==ind:
                    sum_weights+=weights[idx];
            weighted_distances.append((ind,sum_weights));
        #Sorting the distances in descending order based on the sum_weights parameter
        weighted_distances.sort(key=lambda x:x[1],reverse=True);
        #Appending the label corresponding to the highest weighted distance 
        #as the prediction for the current test example
        predictions.append(weighted_distances[0][0]);
    return predictions;

#### Making Predictions

In [ ]:
#Calling the knn function to make predictions for the test examples
predictions=knn(examples,labels,test_examples);
#Printing the predictions
print(predictions)

#### Storing the Results

In [ ]:
#Storing the results in a .out file with appropriate name
with open('19EC30055_P2.out','w') as opf:
    for preds in predictions:
        opf.write('%d ' % preds);